In [1]:
import pandas as pd
import numpy as np

# Load the synthetic data generated previously
platform_df = pd.read_csv('kmrl_platform_data.csv')
depot_df = pd.read_csv('kmrl_depot_data.csv')

# Configuration constants from provided notebooks
TRAIN_CAPACITY = 1000     
SAFETY_BUFFER = 0.20      
MIN_HEADWAY_MIN = 5      # 5 minutes minimum headway for safety/operations
MAX_HEADWAY_MIN = 15     # 15 minutes maximum headway (off-peak)

def optimize_headway_and_coordination(platform_data, depot_data):
    # 1. Aggregate demand at a network level for each time slot to determine total needs
    network_demand = platform_data.groupby(['date', 'time_slot']).agg({
        'entry_count': 'sum',
        'day_type': 'first'
    }).reset_index()

    # 2. Decision Logic for Required Trains
    def calculate_required_trains(row):
        effective_demand = row['entry_count'] * (1 + SAFETY_BUFFER)
        # Calculate trains needed to cover demand
        req = np.ceil(effective_demand / TRAIN_CAPACITY)
        
        # Adjust based on day type (logic from train-induction.ipynb)
        if row['day_type'] == 'Weekend':
            req = max(2, req - 1)
        
        # KMRL constraints (typical fleet size)
        return int(np.clip(req, 4, 15))

    network_demand['required_trains'] = network_demand.apply(calculate_required_trains, axis=1)

    # 3. Calculate Headway (60 mins / trains per hour)
    # Note: 15-min intervals mean we treat 'required_trains' as the active fleet rate
    network_demand['optimized_headway_min'] = (60 / network_demand['required_trains']).clip(MIN_HEADWAY_MIN, MAX_HEADWAY_MIN)

    # 4. Depot Coordination: Match required trains against depot availability
    # We prioritize induction from Muttom Yard (larger capacity)
    depot_pivot = depot_data.pivot_table(
        index=['date', 'time_slot'], 
        columns='depot_name', 
        values='ready_for_induction'
    ).reset_index()

    coordination_df = network_demand.merge(depot_pivot, on=['date', 'time_slot'])

    def determine_induction_source(row):
        needed = row['required_trains']
        if row['Muttom Yard'] >= needed:
            return "Induct from Muttom Yard"
        elif (row['Muttom Yard'] + row['Aluva Depot']) >= needed:
            return "Split Induction (Muttom + Aluva)"
        else:
            return "Alert: Insufficient Ready Trains - Maintenance Override Required"

    coordination_df['depot_action'] = coordination_df.apply(determine_induction_source, axis=1)
    
    return coordination_df

# Run Optimization
optimized_results = optimize_headway_and_coordination(platform_df, depot_df)

# Save and Preview
optimized_results.to_csv('kmrl_headway_optimization.csv', index=False)
print("Optimization Complete. Sample results:")
print(optimized_results[['time_slot', 'entry_count', 'required_trains', 'optimized_headway_min', 'depot_action']].head(10))

Optimization Complete. Sample results:
  time_slot  entry_count  required_trains  optimized_headway_min  \
0  05:30:00         1937                4                   15.0   
1  05:45:00         1911                4                   15.0   
2  06:00:00         1777                4                   15.0   
3  06:15:00         1758                4                   15.0   
4  06:30:00         1786                4                   15.0   
5  06:45:00         1926                4                   15.0   
6  07:00:00         1815                4                   15.0   
7  07:15:00         1617                4                   15.0   
8  07:30:00         1783                4                   15.0   
9  07:45:00         1696                4                   15.0   

              depot_action  
0  Induct from Muttom Yard  
1  Induct from Muttom Yard  
2  Induct from Muttom Yard  
3  Induct from Muttom Yard  
4  Induct from Muttom Yard  
5  Induct from Muttom Yard  
6  Induct